In [ ]:
# Step 1: Install necessary libraries
!pip install -q transformers datasets accelerate bitsandbytes torch evaluate rouge_score sentencepiece bert_score

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from datasets import load_dataset
from kaggle_secrets import UserSecretsClient
import pandas as pd
import random
import evaluate
import warnings
import time

# Suppress warnings to keep the output clean
warnings.filterwarnings("ignore")
from transformers import logging
logging.set_verbosity_error()

In [ ]:
# Step 2: Authenticate with Hugging Face
# This is required to download gated models like Llama 3
try:
    user_secrets = UserSecretsClient()
    hf_token = user_secrets.get_secret("HUGGING_FACE_TOKEN")
except Exception as e:
    print("Could not retrieve Hugging Face token. Please ensure it is stored as a Kaggle secret named 'HUGGING_FACE_TOKEN'.")
    # You can manually paste your token here for local testing if needed:
    # hf_token = "YOUR_HF_TOKEN"
    hf_token = None

In [ ]:
# Step 3: Define Model and Dataset Identifiers
dataset_id = "tmnam20/ViMedAQA"
seed_num = 42
NUM_SAMPLES_INITIAL = 200

# --- ADDED: Boolean toggle for the second randomization ---
# Set to True to get the final 50 samples.
# Set to False to use the initial 200 samples.
ENABLE_SUBSET_SAMPLING = True
NUM_SAMPLES_FINAL = 50

# Step 4: Load and Prepare the Dataset
try:
    dataset = load_dataset(dataset_id, split="train")
    print(f"Dataset loaded successfully! Total samples: {len(dataset)}")

    # --- First Sampling Step: Always get the initial 200 samples ---
    random.seed(seed_num) # for reproducibility
    initial_random_indices = random.sample(range(len(dataset)), NUM_SAMPLES_INITIAL)
    initial_eval_dataset = dataset.select(initial_random_indices)

    print(f"Created an initial random evaluation set with {len(initial_eval_dataset)} samples.")

    # --- ADDED: Conditional second randomization ---
    if ENABLE_SUBSET_SAMPLING:
        print("Subset sampling is ENABLED. Performing second randomization...")
        # Re-seed to ensure this step is also reproducible
        random.seed(seed_num)
        final_random_indices = random.sample(range(len(initial_eval_dataset)), NUM_SAMPLES_FINAL)
        # Final dataset is the smaller, 50-sample subset
        eval_dataset = initial_eval_dataset.select(final_random_indices)
        print(f"Further randomized and reduced the set to a final size of {len(eval_dataset)} samples.")
    else:
        print("Subset sampling is DISABLED.")
        # Final dataset is the larger, 200-sample set
        eval_dataset = initial_eval_dataset
        print(f"Using the initial set of {len(eval_dataset)} samples for evaluation.")

except Exception as e:
    print(f"Failed to load the dataset. Error: {e}")
    eval_dataset = None

In [ ]:
PROMPT_STRATEGIES = {
    "Direct_VI": "Sử dụng Ngữ cảnh sau để trả lời Câu hỏi.",
    "RolePlay_VI": "Bạn là một trợ lý y tế hữu ích. Hãy trả lời Câu hỏi CHỈ dựa vào Ngữ cảnh được cung cấp.",
    "Extract_VI": "Dựa vào Ngữ cảnh sau, trích xuất câu trả lời trực tiếp từ văn bản, không giải thích gì thêm.",
    "Current_Best_VI": (
        "Bạn là một chuyên gia y tế AI với nhiệm vụ trích xuất thông tin chính xác. "
        "Dựa CHỈ vào văn bản trong phần Ngữ cảnh dưới đây, hãy trả lời cho Câu hỏi. "
        "Câu trả lời của bạn phải ngắn gọn, đi thẳng vào vấn đề và không chứa bất kỳ thông tin nào không có trong văn bản. "
        "Không giải thích thêm."
    ),
    "Few_Shot_VI": (
        "Dựa vào các Ví dụ sau đây, hãy trả lời Câu hỏi cuối cùng bằng cách trích xuất thông tin từ Ngữ cảnh được cung cấp.\n\n"
        "--- Ví dụ 1 ---\n"
        "Ngữ cảnh: Thuốc Biviantac được chỉ định để điều trị các trường hợp do tăng tiết acid quá mức như: - Khó tiêu, nóng rát hay đau vùng thượng vị. - Trướng bụng, đầy hơi, ợ nóng, ợ hơi hay ợ chua. - Tăng độ acid, đau rát dạ dày. - Các rối loạn thường gặp trong những bệnh lý loét dạ dày tá tràng, thực quản.\n"
        "Câu hỏi: Biviantac có thể điều trị trướng bụng, đầy hơi không?\n"
        "Câu trả lời: Có, Biviantac có thể điều trị các tình trạng như trướng bụng, đầy hơi, ợ nóng, ợ hơi hay ợ chua.\n\n"
        "--- Ví dụ 2 ---\n"
        "Ngữ cảnh: Thuốc Atorvastatin T.V Pharm được dùng đường uống.\n"
        "Câu hỏi: Tổng hợp các cách dùng hiệu quả để quản lý Atorvastatin T.V Pharm?\n"
        "Câu trả lời: Các cách thức dùng thuốc Atorvastatin T.V Pharm hiệu quả là sử dụng đường uống.\n\n"
        "--- Ví dụ 3 ---\n"
        "Ngữ cảnh: - Buồn nôn, nôn, khó tiêu, khó chịu ở thượng vị, ợ nóng, đau dạ dày, loét dạ dày – ruột. - Mệt mỏi. - Ban, mày đay. - Thiếu máu tan huyết. - Yếu cơ. - Khó thở, sốc phản vệ.\n"
        "Câu hỏi: Các tác dụng phụ thường gặp của thuốc Aspirin 81 là gì?\n"
        "Câu trả lời: Các tác dụng phụ thường gặp của thuốc Aspirin 81 bao gồm buồn nôn, nôn, khó tiêu, khó chịu ở thượng vị, ợ nóng, đau dạ dày, loét dạ dày – ruột.\n\n"
        "--- Ví dụ 4 ---\n"
        "Ngữ cảnh: Các chị em có thể thỉnh thoảng thấy kinh nguyệt ra nhiều hoặc ra máu giữa các kỳ kinh (chảy máu giữa kỳ kinh nguyệt).\n"
        "Câu hỏi: Các chị em có thể gặp tình trạng rong kinh không?\n"
        "Câu trả lời: Có.\n\n"
        "--- Bây giờ, hãy trả lời Câu hỏi sau dựa trên Ngữ cảnh của nó---"
    ),
    "Expert_Persona_VI": (
        "Bạn là một chuyên gia trong lĩnh vực y tế."
        "Dựa trên kiến thức chuyên môn của mình, hãy trả lời Câu hỏi sau CHỈ dựa vào Ngữ cảnh được cung cấp."
    ),
    "Full_VI": (
        "Dựa vào Ngữ cảnh sau, hãy trích xuất câu trả lời **đầy đủ và toàn diện nhất** có thể từ văn bản."
        "Đảm bảo rằng bạn đã bao gồm **tất cả** các điểm có liên quan để trả lời cho câu hỏi."
    ),
    "List_VI": (
        "Từ Ngữ cảnh được cung cấp, hãy **liệt kê tất cả** các thông tin dùng để trả lời cho Câu hỏi."
        "Trình bày câu trả lời một cách ngắn gọn, chỉ bao gồm các điểm được tìm thấy."
    ),
    "No_Verbose_VI": (
        f"TỪ Ngữ cảnh, TRÍCH XUẤT câu trả lời cho Câu hỏi.\n\n"
        f"**QUY TẮC:**\n"
        f"1. CHỈ sử dụng thông tin từ Ngữ cảnh.\n"
        f"2. KHÔNG giải thích các bước của bạn.\n"
        f"3. KHÔNG tự suy luận hoặc thêm bất kỳ thông tin bên ngoài nào.\n"
        f"4. Cung cấp câu trả lời được trích xuất trực tiếp.\n\n"
        f"Dưới đây là Ngữ cảnh và Câu hỏi, hãy đưa câu trả lời TRÍCH XUẤT:"
    ),
}

# Mode 1: Run all strategies defined in PROMPT_STRATEGIES (False)
# Mode 2: Run only the single, specified strategy for a targeted comparison (True)
USE_BEST_PROMPT_ONLY = False
BEST_STRATEGY_NAME = "Extract_VI" # Specify the prompt to use in Mode 2
# "Extract_VI": "Dựa vào Ngữ cảnh sau, trích xuất câu trả lời trực tiếp từ văn bản, không giải thích gì thêm.",

model_ids = [
    # "alpha-ai/LLAMA3-3B-Medical-COT",
    "vilm/vietcuna-3b-v2",
    # "arcee-ai/Arcee-VyLinh",
    # "sail/Sailor-4B",
    "vilm/vinallama-2.7b-chat",
]

In [ ]:
# --- UPDATED: Step 1 - Define Bilingual Prompt Engineering Strategies ---
generation_times = {}

standard_system_prompt = "Bạn là một trợ lý y tế hữu ích. Hãy trả lời Câu hỏi của người dùng CHỈ dựa vào Ngữ cảnh được cung cấp."

def create_prompt_and_get_config(sample, model_id, tokenizer, strategy_name):
    """
    A single, unified function to create a model-specific prompt and return
    the associated answer start tag for parsing.

    Returns:
        tuple: (formatted_prompt_string, answer_start_tag_string)
    """
    context = sample['context']
    question = sample['question']

    # 1. Get the unified instruction text for the chosen strategy
    base_instruction = PROMPT_STRATEGIES.get(strategy_name)
    if not base_instruction:
        raise ValueError(f"Strategy '{strategy_name}' not found in PROMPT_STRATEGIES dictionary.")

    full_instruction_text = f"{base_instruction}\\n\\nNgữ cảnh: {context}\\n\\nCâu hỏi: {question}"

    # 2. Apply the correct, model-specific formatting and define the answer tag
    if model_id == "vilm/vietcuna-3b-v2":
        prompt = f"A chat between a curious user and an artificial intelligence assistant.\\nUSER: {full_instruction_text}\\nASSISTANT:"
        answer_start_tag = "ASSISTANT:"
        return prompt, answer_start_tag

    if model_id == "arcee-ai/Arcee-VyLinh":
        messages = [
            {"role": "system", "content": standard_system_prompt},
            {"role": "user", "content": full_instruction_text}
        ]
        prompt = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        answer_start_tag = "<|im_start|>assistant"
        return prompt, answer_start_tag

    if model_id == "alpha-ai/LLAMA3-3B-Medical-COT":
        messages = [
            {"role": "system", "content": standard_system_prompt},
            {"role": "user", "content": full_instruction_text}
        ]
        prompt = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        answer_start_tag = "<|start_header_id|>assistant<|end_header_id|>"
        return prompt, answer_start_tag

    # https://huggingface.co/sail/Sailor-4B
    if model_id == "sail/Sailor-4B":
        prompt = f"{full_instruction_text}\\n\\nCâu trả lời:"
        answer_start_tag = ""
        return prompt, answer_start_tag
    
    # https://huggingface.co/sail/Sailor-4B-Chat
    if model_id == "sail/Sailor-4B-Chat":
        messages = [
            {"role": "system", "content": standard_system_prompt},
            # IMPORTANT: Sailor examples use role 'question' instead of 'user'
            {"role": "question", "content": full_instruction_text},
        ]
        prompt = tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
        answer_start_tag = "answer:"
        return prompt, answer_start_tag

    if "vilm/vinallama-2.7b" in model_id:
        # system_prompt = "Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác."
        system_prompt = standard_system_prompt
        # Construct the final prompt string using the specified template
        prompt = (
            f"<|im_start|>system\n{system_prompt}<|im_end|>\n"
            f"<|im_start|>user\n{full_instruction_text}<|im_end|>\n"
            f"<|im_start|>assistant"
        )

        # The answer_start_tag is the exact string that precedes the model's response
        answer_start_tag = "<|im_start|>assistant"

        return prompt, answer_start_tag

    # Nothing matches
    return "", ""

In [ ]:
# Step 5: Generate Answers from Each Model
all_generated_answers = {}

if eval_dataset and hf_token:
    # Wrap each answer in a list to create the required List[List[str]] structure
    ground_truth_answers = [[sample['answer']] for sample in eval_dataset] 
    questions = [sample['question'] for sample in eval_dataset]

    wide_results = []
    for i, sample in enumerate(eval_dataset):
        wide_results.append({
            "Sample_ID": i,
            "Question": sample['question'],
            "Context": sample['context'],
            "Ground_Truth_Answer": ground_truth_answers[i][0]
        })
    
    # Loop through each model to generate answers
    for model_id in model_ids:
        print("\n" + "="*50)
        print(f"Loading model: {model_id}")
        print("="*50)

        model, tokenizer, text_generator = None, None, None

        try:
            # Load the tokenizer and model with 4-bit quantization to save memory
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16,
                bnb_4bit_use_double_quant=False,
            )
            tokenizer = AutoTokenizer.from_pretrained(model_id, token=hf_token)
            model = AutoModelForCausalLM.from_pretrained(
                model_id,
                token=hf_token,
                quantization_config=bnb_config, # <-- PASS THE CONFIG OBJECT HERE
                device_map="auto",
                trust_remote_code=True
            )

            # Set up the text generation pipeline
            text_generator = pipeline(
                "text-generation",
                model=model,
                tokenizer=tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
            )

            prompt_variations = []
            if USE_BEST_PROMPT_ONLY:
                # Mode 2: Run only the single specified prompt strategy
                prompt_variations = [BEST_STRATEGY_NAME]
                print(f"Mode: Best Prompt Only. Running with the fair strategy: '{BEST_STRATEGY_NAME}'")
            else:
                # Mode 1: Run all defined prompt strategies for a full exploration
                prompt_variations = list(PROMPT_STRATEGIES.keys())
                print(f"Mode: Exploration. Running all {len(prompt_variations)} strategies: {prompt_variations}")

                
            for prompt_name in prompt_variations:
                print(f"\n--- Prompt Strategy: {prompt_name} ---")
                # Create a unique key for each result set
                result_key = f"{model_id} ({prompt_name})"
                
                # Generate prompts and get model config from our new unified function
                prompt_data = [create_prompt_and_get_config(sample, model_id, tokenizer, prompt_name) for sample in eval_dataset]
                prompts = [p[0] for p in prompt_data]
                # if model_id == "sail/Sailor-4B-Chat":
                #     prompts = ["Hãy cho tôi một giới thiệu ngắn gọn về mô hình ngôn ngữ lớn."]
                answer_start_tag = prompt_data[0][1] # Get tag from the first generated prompt
                if prompts == "" and answer_start_tag == "":
                    print("No model matches")
                    break

                start_time = time.time()

                # print(f"Generating answers for {len(prompts)} prompts using {model_id} with '{prompt_name}' strategy...")
                # Generate answers for the entire batch
                generated_outputs_batch = text_generator(
                    prompts,
                    max_new_tokens=256,
                    do_sample=False,
                    eos_token_id=tokenizer.eos_token_id,
                    pad_token_id=tokenizer.eos_token_id,
                )

                end_time = time.time()
                generation_time = end_time - start_time
                generation_times[result_key] = generation_time # Lưu thời gian đã đo
    
                # Extract the clean answers
                model_answers = []
                
                for i, output in enumerate(generated_outputs_batch):
                    generated_text = output[0]['generated_text']
                    # Check if the tag is not empty AND exists in the text before splitting
                    if answer_start_tag and answer_start_tag in generated_text:
                        clean_answer = generated_text.split(answer_start_tag)[-1].strip()
                    else:
                        # Fallback for base models or if the tag isn't found
                        clean_answer = generated_text.replace(prompts[i], "").strip()
                    model_answers.append(clean_answer)
    
                # Use the unique result_key to store the answers
                all_generated_answers[result_key] = model_answers
                # print(f"Successfully generated answers for {result_key}.")

                # --- Thêm các câu trả lời đã tạo như một cột mới vào cấu trúc "rộng" ---
                # Tên cột sẽ là nhãn của prompt, ví dụ: "Answer_vilm/vietcuna-3b-v2 (Current_Best_VI)"
                answer_column_name = f"Answer_{result_key}"
                
                # Lặp qua các câu trả lời và thêm chúng vào đúng hàng trong wide_results
                for i in range(len(model_answers)):
                    wide_results[i][answer_column_name] = model_answers[i]

                print(f"Time for generating answer: {generation_time:.2f} seconds.")

        except Exception as e:
            print(f"An error occurred while processing {model_id}: {e}")
        finally:
            # Check if variables were successfully created before deleting
            if model is not None: del model
            if tokenizer is not None: del tokenizer
            if text_generator is not None: del text_generator
            torch.cuda.empty_cache()

            import gc
            gc.collect()

else:
    print("Skipping generation due to issues with the dataset or Hugging Face token.")

In [ ]:
# --- Bước 5.5 - Lưu tất cả các câu trả lời đã tạo vào một tệp CSV ---
if wide_results:
    print("\n" + "="*50)
    print("Đang lưu kết quả định dạng rộng vào tệp CSV...")
    print("="*50)
    
    # Chuyển đổi danh sách kết quả thành một DataFrame của pandas
    results_df_wide = pd.DataFrame(wide_results)
    
    # Chỉ định đường dẫn tệp đầu ra trong thư mục làm việc của Kaggle
    output_file_path = "/kaggle/working/results.csv"
    
    # Lưu DataFrame vào tệp CSV
    results_df_wide.to_csv(output_file_path, index=False, encoding='utf-8-sig')
    
    print(f"Hoàn tất! Đã lưu {len(results_df_wide)} hàng (mẫu) vào tệp:")
    print(output_file_path)
    
    # Hiển thị 5 hàng đầu tiên của tệp đã lưu để xem trước
    # Bạn sẽ thấy các cột câu trả lời khác nhau cho mỗi chiến lược prompt
    display(results_df_wide.head())
    
else:
    print("\nKhông có kết quả nào để lưu.")

In [ ]:
# Step 6: Evaluate the Generated Answers
if all_generated_answers:
    # Load all the metrics we need
    rouge_metric = evaluate.load('rouge')
    bleu_metric = evaluate.load('bleu')
    meteor_metric = evaluate.load('meteor')
    bertscore_metric = evaluate.load('bertscore')

    evaluation_results = []

    print("\n" + "="*50)
    print("Calculating Evaluation Metrics")
    print("="*50)

    for result_key, predictions in all_generated_answers.items():
        print(f"\n--- Evaluating {result_key} ---")
    
        # Check for empty predictions to prevent ZeroDivisionError in BLEU ---
        # The `any()` function returns False if all strings in the list are empty.
        if not any(predictions):
            print(f"  WARNING: Model & Prompt Strategy '{result_key}' produced empty answers for all samples. Assigning all metric scores to 0.")
            result_row = {
                "Model & Prompt Strategy": result_key,
                "ROUGE-L": 0.0,
                "BLEU": 0.0,
                "METEOR": 0.0,
                "BERTScore-F1": 0.0
            }
            evaluation_results.append(result_row)
            # Use `continue` to skip the rest of the loop and move to the next model
            continue
    
        # If predictions are valid, compute metrics as normal
        rouge_scores = rouge_metric.compute(predictions=predictions, references=ground_truth_answers)
        bleu_scores = bleu_metric.compute(predictions=predictions, references=ground_truth_answers)
        meteor_scores = meteor_metric.compute(predictions=predictions, references=ground_truth_answers)
        bertscore_scores = bertscore_metric.compute(predictions=predictions, references=ground_truth_answers, lang="vi")
    
        # Store results (this part is the same as before)
        result_row = {
            "Model & Prompt Strategy": result_key,
            "ROUGE-L": round(rouge_scores['rougeL'], 4),
            "BLEU": round(bleu_scores['bleu'], 4),
            "METEOR": round(meteor_scores['meteor'], 4),
            "BERTScore-F1": round(sum(bertscore_scores['f1']) / len(bertscore_scores['f1']), 4),
            "Generation Time (s)": round(generation_times.get(result_key, 0), 2), # Lấy thời gian đã lưu
        }
        evaluation_results.append(result_row)

    # Step 7: Display Results
    results_df = pd.DataFrame(evaluation_results)
    # Sort for better comparison
    results_df = results_df.sort_values(by="BERTScore-F1", ascending=False).reset_index(drop=True)
    print("\n--- Comparative Evaluation Results ---")
    display(results_df)

else:
    print("\nNo answers were generated. Skipping evaluation.")